In [10]:
import warnings 
warnings.filterwarnings('ignore')

# lire films

In [11]:
import pandas as pd 
movies = pd.read_csv('movielens/movies.csv', header=0)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# nombre des films

In [12]:
movies.shape

(9742, 3)

# lire les notes

In [13]:
ratings = pd.read_csv('movielens/ratings.csv', header=0)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# nombre des notes

In [14]:
ratings.shape

(100836, 4)

# faire test et former dataa a travers annee de film par example 20% data test and 80% former data

# vérifier si il y a un column vide dans films

In [15]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

# fait le test sur tout qui plus grand que la date 2015

# diviser la date par le titre

In [16]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')
movies['year'] = movies['year'].fillna('1999')

movies.head() 

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


# divisier data a test apres 2015 et former avant 2015

In [17]:
threshold_date = '2015'
ids = movies[movies['year'] < threshold_date]['movieId'].values

training_data = ratings[ratings['movieId'].isin(ids)]
print(f'Training data size :{training_data.shape}')
testing_data = ratings[~ratings['movieId'].isin(ids)]
print(f'Testing data size :{testing_data.shape}')

Training data size :(98411, 4)
Testing data size :(2425, 4)


In [18]:
def get_favorite_movies(user_id, ratings_df):
    favorites = ratings_df[(ratings_df['userId'] == user_id) & (ratings_df['rating'] >= 3.5)].sort_values(by='rating', ascending=False)['movieId']
    return set(favorites if type(favorites) == pd.Series else [favorites])

# Recall et Precision 
recall = recommandé et il aime / nombre de recommende
precision = recommandé et il aime / tous que recommandé qu'il aime

mais il y a un pproblem 
quand il donne 1000 recommende par example il fais seulement sur un nombre aleatoire 

et on fais une chose c'est precision@k et recall@k 
par example:
5 recommendation 10111 
et prescision@k k=3 il vas prend 2 qui recommende et les recommendation qu'il aime par user sur nombre k  donc 2/3 
et recall@k k=3 il vas prend 2 qui recommende et les recommendation qu'il aime par user sur nombre de tous les recommendations qu'il aime 4 donc 2/4

In [19]:
class ModelEvaluator:
    def __init__(self, training_data, testing_data, threshold=3.5):
        self.training_data = training_data
        self.testing_data = testing_data
        self.threshold = threshold 
    
    def evaluate_model_for_user(self, model, user_id):
        # Obtenir les films favoris dans le test
        favorites_in_test = get_favorite_movies(user_id, self.testing_data)
        
        # Obtenir les recommandations du modèle pour l'utilisateur donné
        # à travers ses films favoris, en utilisant les données d'entraînement
        person_recs_df = model.recommend_items(user_id, items_to_ignore=get_favorite_movies(user_id, self.training_data))
        
        # Filtrer les recommandations en fonction du seuil et trier par note prédite
        person_recs_df = person_recs_df[person_recs_df['predicted_rating'] >= self.threshold].sort_values(by='predicted_rating', ascending=False)
        
        # Calculer le nombre de films réellement recommandés qui sont des favoris dans le test
        true_relevant = person_recs_df[person_recs_df['movieId'].isin(favorites_in_test)].shape[0]
        
        # Sélectionner les 5 premières recommandations
        top_5_recommended = person_recs_df.head(5)
        
        # Sélectionner les 10 premières recommandations
        top_10_recommended = person_recs_df.head(10)
        
        # Calculer les nombres de hits@5 et hits@10
        hits_at_5_count = top_5_recommended[top_5_recommended['movieId'].isin(favorites_in_test)].shape[0]
        hits_at_10_count = top_10_recommended[top_10_recommended['movieId'].isin(favorites_in_test)].shape[0]
        
        # Calculer la précision@5 et le rappel@5
        precision_at_5 = hits_at_5_count / top_5_recommended.shape[0] if top_5_recommended.shape[0] != 0 else 1
        recall_at_5 = hits_at_5_count / true_relevant if true_relevant != 0 else 1
        
        # Calculer la précision@10 et le rappel@10
        precision_at_10 = hits_at_10_count / top_10_recommended.shape[0] if top_10_recommended.shape[0] != 0 else 1
        recall_at_10 = hits_at_10_count / true_relevant if true_relevant != 0 else 1
        
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'recommended@5_count': top_5_recommended.shape[0],
            'recommended@10_count': top_10_recommended.shape[0],
            'relevents': true_relevant,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10,
            'precision@5': precision_at_5,
            'precision@10': precision_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        users_metrics = []
        users_ids = list(set(self.testing_data['userId'].values))
        for idx, user_id in enumerate(users_ids):
            metrics = self.evaluate_model_for_user(model, user_id)
            metrics['user_id'] = user_id
            users_metrics.append(metrics)
        print('%d users processed' % idx)
        
        detailed_results_df = pd.DataFrame(users_metrics).sort_values('hits@5_count', ascending=False)

        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['relevents'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['relevents'].sum())
        
        global_precision_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['recommended@5_count'].sum())
        global_precision_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['recommended@10_count'].sum())
        
        global_metrics = {
            'modelName': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10,
            'precision@5': global_precision_at_5,
            'precision@10': global_precision_at_10
        }
        return global_metrics, detailed_results_df

model_evaluator = ModelEvaluator(training_data, testing_data)


# Popularity Recommender  
la moyenne des notes par rapport les nombre des utilisateurs qui fais les notes
(v/(v+m)*r)+(v/(v+m)*c)
v nombre des notes dans le film 
m note minimale requise pour être répertorié
r la moyenne des notes
c la moyenne des notes sur l'ensemble des films

In [20]:
popularity = ratings.groupby('movieId').agg({'rating':['mean', 'count']}).reset_index()
popularity.columns = ['movieId','ratings_mean', 'ratings_count']
popularity.sort_values(by='ratings_mean', ascending=False)

,movieId,ratings_mean,ratings_count
7638,88448,5.0,1
8089,100556,5.0,1
9065,143031,5.0,1
9076,143511,5.0,1
9078,143559,5.0,1
...,...,...,...
9253,157172,0.5,1
7536,85334,0.5,1
6486,53453,0.5,1
5200,8494,0.5,1


In [21]:
class PopularityRecommender:
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularities_df):
        self.popularities_df = popularities_df
    
    def get_model_name(self):
        return self.MODEL_NAME
    
    def weighted_rating(self, x, m, C):
        V = x['ratings_count']
        R = x['ratings_mean']
        return (V/(V+m)*R) + (m/(m+V)* C)
                
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        C = self.popularities_df['ratings_mean'].mean()
        
        self.popularities_df['predicted_rating'] = self.popularities_df.apply(lambda x: self.weighted_rating(x,3.5,C), axis=1) 
        
        recommendations_df = self.popularities_df[~self.popularities_df['movieId'].isin(items_to_ignore)].sort_values('predicted_rating', ascending=False).head(topn)
        return recommendations_df
    
popularity_model = PopularityRecommender(popularity)
        

In [22]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\n Global metrics: \n%s'%pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
124 users processed

 Global metrics: 
{'modelName': 'Popularity', 'recall@5': 0.8888888888888888, 'recall@10': 1.0, 'precision@5': 0.0128, 'precision@10': 0.0072}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
16,1,1,5,10,1,1.0,1.0,0.2,0.1,25
70,1,1,5,10,1,1.0,1.0,0.2,0.1,212
94,1,1,5,10,1,1.0,1.0,0.2,0.1,338
11,1,1,5,10,1,1.0,1.0,0.2,0.1,18
72,1,1,5,10,1,1.0,1.0,0.2,0.1,233
89,1,1,5,10,1,1.0,1.0,0.2,0.1,318
109,1,2,5,10,2,0.5,1.0,0.2,0.2,414
68,1,1,5,10,1,1.0,1.0,0.2,0.1,209
90,0,0,5,10,0,1.0,1.0,0.0,0.0,319
81,0,0,5,10,0,1.0,1.0,0.0,0.0,279


In [23]:
popularity_model.recommend_items(5)

,movieId,ratings_mean,ratings_count,predicted_rating
277,318,4.429022,317,4.416283
9600,177593,4.750000,8,4.297267
840,1104,4.475000,20,4.294407
659,858,4.289062,192,4.270683
796,1041,4.590909,11,4.270246
2224,2959,4.272936,218,4.256969
882,1178,4.541667,12,4.252811
2579,3451,4.545455,11,4.235763
921,1221,4.259690,129,4.233348
602,750,4.268041,97,4.233021


In [24]:
popularity_model.recommend_items(10)

,movieId,ratings_mean,ratings_count,predicted_rating
277,318,4.429022,317,4.416283
9600,177593,4.750000,8,4.297267
840,1104,4.475000,20,4.294407
659,858,4.289062,192,4.270683
796,1041,4.590909,11,4.270246
2224,2959,4.272936,218,4.256969
882,1178,4.541667,12,4.252811
2579,3451,4.545455,11,4.235763
921,1221,4.259690,129,4.233348
602,750,4.268041,97,4.233021


In [25]:
ids = popularity_model.recommend_items(10)['movieId'].values
movies[movies['movieId'].isin(ids)].values

array([[318, 'Shawshank Redemption, The (1994)', 'Crime|Drama', '1994'],
       [750,
        'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
        'Comedy|War', '1964'],
       [858, 'Godfather, The (1972)', 'Crime|Drama', '1972'],
       [1041, 'Secrets & Lies (1996)', 'Drama', '1996'],
       [1104, 'Streetcar Named Desire, A (1951)', 'Drama', '1951'],
       [1178, 'Paths of Glory (1957)', 'Drama|War', '1957'],
       [1221, 'Godfather: Part II, The (1974)', 'Crime|Drama', '1974'],
       [2959, 'Fight Club (1999)', 'Action|Crime|Drama|Thriller', '1999'],
       [3451, "Guess Who's Coming to Dinner (1967)", 'Drama', '1967'],
       [177593, 'Three Billboards Outside Ebbing, Missouri (2017)',
        'Crime|Drama', '2017']], dtype=object)

# Content-Based Filtering model 


# utiliser genres comme content

In [26]:
#get all unique genres
genres = list(set([x for genres in movies['genres'].values for x in genres.split('|')]))
genres

['Children',
 '(no genres listed)',
 'Romance',
 'Drama',
 'Documentary',
 'Mystery',
 'Action',
 'IMAX',
 'Comedy',
 'Musical',
 'Film-Noir',
 'War',
 'Adventure',
 'Horror',
 'Fantasy',
 'Thriller',
 'Western',
 'Sci-Fi',
 'Crime',
 'Animation']

In [27]:
expanded_movies_df = movies.copy()
for g in genres:
    expanded_movies_df[g] = [0 if g in genres.split('|') else 1 for genres in movies['genres'].values]
expanded_movies_df.head()
    

,movieId,title,genres,year,Children,(no genres listed),Romance,Drama,Documentary,Mystery,...,Film-Noir,War,Adventure,Horror,Fantasy,Thriller,Western,Sci-Fi,Crime,Animation
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,0,1,1,1,1,1,...,1,1,0,1,0,1,1,1,1,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,0,1,1,1,1,1,...,1,1,0,1,0,1,1,1,1,1
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,1,1,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
4,5,Father of the Bride Part II (1995),Comedy,1995,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [28]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def normalize(value, old_max, old_min,new_max=5.0, new_min=0.0):
    old_range = (old_max - old_min)
    new_range = (new_max - new_min)
    return (((value - old_min) * new_range) / old_range) + new_min


In [29]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self, items_df, training_data, testing_data):
        self.items_df = items_df
        self.features_names = items_df.columns[4:]
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_item_profile(self, item_id):
        #take vector
        item_profile = self.items_df[self.items_df['movieId'] == item_id][self.features_names]
        return item_profile
    def get_items_profiles(self, ids):
        item_profiles = self.items_df[self.items_df['movieId'].isin(ids)][self.features_names].values
        return item_profiles
    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['userId'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['movieId'].values)
        
        user_items_ratings = np.array(user_df['rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        return user_profile
    
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        cosine_similarities = cosine_similarity(user_profile,self.items_df[self.features_names].values)
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(self.items_df.loc[i, 'movieId'], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'predicted_rating']).head(topn)
        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x : normalize(x, 1.0, 0.0))
        return recommendations_df
content_based_recommender_model = ContentBasedRecommender(expanded_movies_df, training_data, testing_data)

In [30]:
print('Evaluating Content Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content Based Filtering model...
124 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 1.0, 'recall@10': 1.0, 'precision@5': 0.0032, 'precision@10': 0.0016}


,hits@5_count,hits@10_count,recommended@5_count,recommended@10_count,relevents,recall@5,recall@10,precision@5,precision@10,user_id
53,2,2,5,10,2,1.0,1.0,0.4,0.2,111
0,0,0,5,10,0,1.0,1.0,0.0,0.0,2
78,0,0,5,10,0,1.0,1.0,0.0,0.0,256
91,0,0,5,10,0,1.0,1.0,0.0,0.0,326
90,0,0,5,10,0,1.0,1.0,0.0,0.0,319
89,0,0,5,10,0,1.0,1.0,0.0,0.0,318
88,0,0,5,10,0,1.0,1.0,0.0,0.0,306
87,0,0,5,10,0,1.0,1.0,0.0,0.0,305
86,0,0,5,10,0,1.0,1.0,0.0,0.0,298
85,0,0,5,10,0,1.0,1.0,0.0,0.0,296


In [32]:
tags = pd.read_csv('movielens/tags.csv',header=0)
tags.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [33]:
def check_name(x):
    words = x.split()
    return all(x[0].isupper() and x[1:].islower() for x in words if len(x) > 1)

def clean(x):
    if x.istitle() or check_name(x):
        return x.replace(" ", "").lower()
    else:
        return x.lower().strip()


In [34]:
tags['tag'] = tags['tag'].apply(lambda x: clean(x))
tags.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,boxing story,1445715207
4,2,89774,mma,1445715200
5,2,89774,tomhardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,leonardo dicaprio,1445715051
8,2,106782,martinscorsese,1445715056
9,7,48516,way too long,1169687325


In [35]:
expended_movies_df = movies.copy()
movies_ids = expended_movies_df['movieId'].values.tolist()
expended_movies_df['soup'] = [" ".join(tags[tags['movieId'] == id]['tag'].values.tolist()) for id in movies_ids]


In [36]:
expended_movies_df.head(10)

,movieId,title,genres,year,soup
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game robinwilliams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake
5,6,Heat (1995),Action|Crime|Thriller,1995,
6,7,Sabrina (1995),Comedy|Romance,1995,remake
7,8,Tom and Huck (1995),Adventure|Children,1995,
8,9,Sudden Death (1995),Action,1995,
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,


In [37]:
expended_movies_df.loc[8, 'soup']

''

In [38]:
expended_movies_df[expended_movies_df['soup'] != ''].shape

(1572, 5)

In [39]:
def fill_empty_tag(x):
    if x['soup'] == '' :
        return " ".join(x['genres'].lower().split('|'))
    return x['soup']

expended_movies_df['soup'] = expended_movies_df.apply(lambda x: fill_empty_tag(x), axis=1)
expended_movies_df.head(10)

,movieId,title,genres,year,soup
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,fantasy magic board game robinwilliams game
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,comedy drama romance
4,5,Father of the Bride Part II (1995),Comedy,1995,pregnancy remake
5,6,Heat (1995),Action|Crime|Thriller,1995,action crime thriller
6,7,Sabrina (1995),Comedy|Romance,1995,remake
7,8,Tom and Huck (1995),Adventure|Children,1995,adventure children
8,9,Sudden Death (1995),Action,1995,action
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,action adventure thriller


In [40]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(expended_movies_df['soup'])
count_matrix.shape

(9742, 1586)

In [41]:
class ContentBasedRecommender:
    MODEL_NAME = 'Content-Based'
    def __init__(self, items_ids, items_matrix, training_data, testing_data):
        self.items_ids = items_ids
        self.items_matrix = items_matrix
        self.training_data = training_data
        self.testing_data = testing_data
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def get_item_profile(self, item_id):
        idx = self.items_ids.index(item_id)
        return self.items_matrix[idx].toarray().reshape(-1)
    
    def get_items_profiles(self, ids):
        item_profiles = np.array([self.get_item_profile(x) for x in ids])
        return item_profiles
    def build_users_profile(self, user_id):
        user_df = self.training_data[self.training_data['userId'] == user_id]
        user_items_profiles = self.get_items_profiles(user_df['movieId'].values)
        
        user_items_ratings = np.array(user_df['rating'].values).reshape(-1,1)
        user_profile = np.sum(np.multiply(user_items_profiles, user_items_ratings), axis=0) / np.sum(user_items_ratings)
        return user_profile
    
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        user_profile = self.build_users_profile(user_id).reshape(1, -1)
        cosine_similarities = cosine_similarity(user_profile,self.items_matrix.toarray())
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        similar_items = sorted([(self.items_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'predicted_rating']).head(topn)
        recommendations_df['predicted_rating'] = recommendations_df['predicted_rating'].apply(lambda x : normalize(x, 1.0, 0.0))
        return recommendations_df
content_based_recommender_model = ContentBasedRecommender(expanded_movies_df['movieId'].values.tolist(), count_matrix, training_data, testing_data)

In [ ]:
print('Evaluating Content Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content Based Filtering model...


In [ ]:
content_based_recommender_model.recommend_items(8)

In [ ]:
ids = content_based_recommender_model.recommend_items(8)['movieId'].values
movies[movies['movieId'].isin(ids)].values

In [ ]:
ids = content_based_recommender_model.recommend_items(60)['movieId'].values
movies[movies['movieId'].isin(ids)].values

In [ ]:
ids = content_based_recommender_model.recommend_items(60)['movieId'].values
movies[movies['movieId'].isin(ids)].values

# Collaborative filtering

# - Collaborative filtering recommends items based on the preferences of users who have similar tastes. It does not rely on the attributes of items but rather on the behavior of users.
	- There are two main types of collaborative filtering:
		- User-based collaborative filtering: Recommends items that similar users have liked or interacted with.(from high rate film fo              r other users, was watch the same film)
			=> based on the ratings or interactions of other users.
		- Item-based collaborative filtering: Recommends items that are similar to items that the user has liked or interacted with in             the past.(from charac or genrder ers of film)
			=> based solely on the characteristics or attributes of the items themselvess, not on other users' ratings.
	- Collaborative filtering is effective in making recommendations even when the attributes of items are not known or available.

![Sample Image](imgs/svd.png)

In a movie recommendation system, SVD can help identify latent factors (e.g., genre preferences) in user ratings. The reconstructed matrix can be used to predict user ratings for movies they haven't seen based on the discovered patterns.

In [ ]:
# Convert ratings dataframe into a pivot table where rows represent users, columns represent movies,
# and the values represent ratings. Missing values (i.e., when a user hasn't rated a movie) are filled with 0.
users_items_pivot_matrix_df = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
users_items_pivot_matrix_df.head(10)

In [ ]:
# Convert the pivot table dataframe into a NumPy array for matrix factorization.
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix

### Perform matrix factorization using Singular Value Decomposition (SVD) to decompose the user-item interaction matrix#####  into three matrices: U (user matrix), sigma (diagonal matrix containing singular values), and Vt (item matrix). # The parameter k specifies the number of latent factors to consider.

In [ ]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [ ]:
# Check the shape of the U matrix (user matrix)
U.shape

In [ ]:
# Check the shape of the Vt matrix (item matrix)
Vt.shape

In [ ]:
sigma = np.diag(sigma)
sigma.shape

### Calculate all user predicted ratings by taking the dot product of U, sigma, and Vt matrices.

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

### Create a DataFrame to store the predicted ratings for all users and items.

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=users_items_pivot_matrix_df.columns, index = users_items_pivot_matrix_df.index)

In [ ]:

preds_df.head(10)

### Normalize the predicted ratings to a specified range.

In [ ]:
preds_df = preds_df.apply(lambda x: normalize(x, all_user_predicted_ratings.max(), all_user_predicted_ratings.min()))

In [ ]:
preds_df.head(10)

### Define a class for Collaborative Filtering Recommender.

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, predictions_df):
        self.predictions_df = predictions_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        sorted_user_predictions = self.predictions_df.loc[user_id].sort_values(ascending=False)
        recommendations = {'movieId': sorted_user_predictions.index, 'predicted_rating':sorted_user_predictions.values}
        recommendations_df = pd.DataFrame(recommendations)
        recommendations_df = recommendations_df[~recommendations_df['movieId'].isin(items_to_ignore)].sort_values('predicted_rating', ascending = False).head(topn)
        return recommendations_df
        
# Evaluate the collaborative filtering model using the ModelEvaluator class.
cf_recommender_model = CFRecommender(preds_df)

### This defines a class CFRecommender for collaborative filtering recommendation. It contains methods for getting the model name and recommending items for a given user.

#### instance of the <font color='#87CEEB'>CFRecommender</font>  class, evaluates the collaborative filtering model using the ModelEvaluator class, and prints the evaluation results.

In [ ]:
# Evaluate the collaborative filtering model using the ModelEvaluator class.
model_evaluator = ModelEvaluator(training_data, testing_data, 2.5)

# Print evaluation results for the collaborative filtering model.
print("Evaluating collaborative Filtering (SVD MATRIX FACTORIZATION) model ...")
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\n Global metrics: \n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

### Recommend items for a specific user (e.g., user with ID 5).

In [ ]:
cf_recommender_model.recommend_items(5)

### Recommend items for a specific user (with ID 6).

In [ ]:
cf_recommender_model.recommend_items(6)